# Shift Scheduling Demo (D-Wave)

This notebook compares two approaches to QUBO formulation of constraints using the **sparse-qubo** library:

- **NetworkType.NAIVE**: Conventional monolithic formulation (equality constraints over all variables encoded directly as QUBO)
- **NetworkType.DIVIDE_AND_CONQUER**: Constraint decomposition (uses divide and cpmquer networks with auxiliary variables to build a sparser QUBO)

Metrics compared:

- **BQM structure**: Number of variables and of quadratic (2-body) interactions
- **D-Wave execution**: Physical qubits after embedding, chain strength, chain break rate, and solution energy / feasibility

The central function `create_scheduling_problem_bqm` illustrates typical usage of the library: adding "k selected" constraints per row/column via `sparse_qubo.create_constraint_dwave`.

## Environment setup and imports

Add the project root to the path and import sparse-qubo plus shift-scheduling helpers (`create_scheduling_problem_bqm`, `check_feasibility`, etc.).

In [1]:
import os
import sys
from pathlib import Path

# Add project root to path (assumes running from examples/shift_scheduling)
current_dir = Path(os.getcwd())
project_root = current_dir.parent.parent
sys.path.append(str(project_root))

import sparse_qubo  # noqa: E402
from examples.common import calculate_chain_break_rate, sampling_with_dwave  # noqa: E402
from examples.shift_scheduling.problem import (  # noqa: E402
    check_feasibility,
    check_incompatible_pairs,
    create_incompatible_pairs,
    create_scheduling_cost_matrix,
    create_scheduling_problem_bqm,
)

## Problem parameters

- `num_days` / `num_workers`: Number of days and workers (rows and columns of the schedule matrix)
- `shifts_per_day` / `shifts_per_worker`: Shifts per day and per worker (row-sum and column-sum constraint values)
- `num_incompatible_pairs`: Number of worker pairs that must not be assigned the same day (generated at random)
- `naive_penalty_factor` / `dc_penalty_factor`: Penalty weight for constraint violation (tunable per formulation)

In [2]:
num_days = 6
num_workers = 2 * num_days  # 12 workers
num_incompatible_pairs = 5
shifts_per_day = 2
shifts_per_worker = 1
naive_penalty_factor = 20  # You should tune this value
dc_penalty_factor = 15  # You should tune this value
seed = 42

## Problem data

- **Incompatible pairs** (`incompatible_pairs`): Set of worker pairs that must not be assigned the same day
- **Cost matrix** (`cost_matrix`): Cost per (day, worker); lower is better. The objective is expressed as these linear terms

In [3]:
incompatible_pairs = create_incompatible_pairs(num_workers, num_incompatible_pairs, seed=seed)
cost_matrix = create_scheduling_cost_matrix(num_days, num_workers, seed=seed)

print("Incompatible pairs:", incompatible_pairs)
print("Cost matrix:\n", cost_matrix)

Incompatible pairs: {(0, 1), (3, 4), (2, 7), (6, 10), (8, 11)}
Cost matrix:
 [[-2. -1. -3. -1. -1. -4. -3. -3. -3. -1. -2. -3.]
 [-1. -4. -2. -4. -2. -1. -5. -2. -4. -1. -2. -5.]
 [-5. -3. -3. -4. -2. -2. -3. -2. -2. -5. -3. -1.]
 [-3. -1. -5. -4. -2. -5. -2. -4. -4. -5. -4. -1.]
 [-4. -2. -2. -2. -2. -1. -3. -5. -2. -4. -2. -4.]
 [-4. -2. -1. -4. -4. -2. -4. -4. -2. -2. -5. -1.]]


## QUBO formulation: using `create_scheduling_problem_bqm`

This is **typical sparse-qubo usage**. Inside `create_scheduling_problem_bqm`:

1. **Row constraints** (exactly `shifts_per_day` workers per day): Call `sparse_qubo.create_constraint_dwave(..., ConstraintType.EQUAL_TO, network_type, c1=shifts_per_day)` for each row’s variables and add the resulting BQM
2. **Column constraints** (each worker assigned exactly `shifts_per_worker` days): Same with `create_constraint_dwave` per column
3. **Incompatible pairs**: Add quadratic terms that penalize assigning both workers of a pair on the same day
4. Multiply the constraint part by `penalty_factor`, then add linear terms from the cost matrix

We build BQMs for both NAIVE and DIVIDE_AND_CONQUER and compare them in the following cells.

In [4]:
# Conventional: encode constraints as a single linear equality QUBO (fewer variables, denser quadratic terms)
naive_bqm = create_scheduling_problem_bqm(
    sparse_qubo.NetworkType.NAIVE,
    num_days,
    num_workers,
    shifts_per_day,
    shifts_per_worker,
    incompatible_pairs,
    cost_matrix,
    penalty_factor=naive_penalty_factor,
)
# Decomposed: constraints via divide and conquer networks (auxiliary variables yield sparser quadratic terms)
dc_bqm = create_scheduling_problem_bqm(
    sparse_qubo.NetworkType.DIVIDE_AND_CONQUER,
    num_days,
    num_workers,
    shifts_per_day,
    shifts_per_worker,
    incompatible_pairs,
    cost_matrix,
    penalty_factor=dc_penalty_factor,
)

## BQM structure comparison

Compare the number of logical variables and of quadratic (2-body) interactions. DIVIDE_AND_CONQUER increases variables via auxiliaries but can reduce interaction count, which may help embedding and solving.

In [5]:
n_inter = naive_bqm.num_interactions
dc_inter = dc_bqm.num_interactions
reduction = (1 - dc_inter / n_inter) * 100

print(f"{'Metric':<20} | {'Naive':<10} | {'D&C':<10}")
print("-" * 46)
print(f"{'Num Variables':<20} | {len(naive_bqm.variables):<10} | {len(dc_bqm.variables):<10}")
print(f"{'Num Interactions':<20} | {naive_bqm.num_interactions:<10} | {dc_bqm.num_interactions:<10}")
print(f"Interactions Reduced: {reduction:.1f}% ({n_inter} -> {dc_inter})")

Metric               | Naive      | D&C       
----------------------------------------------
Num Variables        | 72         | 720       
Num Interactions     | 576        | 558       
Interactions Reduced: 3.1% (576 -> 558)


## Using the D-Wave quantum annealer

Import the D-Wave API, verify solver connectivity, then set annealing parameters and sample with both formulations.

In [6]:
import dwave.cloud
import dwave.system  # For EmbeddingComposite, DWaveSampler

In [7]:
# Create client from config (e.g. ~/.config/dwave/dwave.conf) and list available solvers
client = dwave.cloud.Client.from_config()
print(client.get_solvers())

[BQMSolver(name='hybrid_binary_quadratic_model_version2p'), DQMSolver(name='hybrid_discrete_quadratic_model_version1p'), CQMSolver(name='hybrid_constrained_quadratic_model_version1p'), NLSolver(name='hybrid_nonlinear_program_version1p'), StructuredSolver(name='Advantage_system4.1', graph_id='01d07086e1'), StructuredSolver(name='Advantage_system6.4', graph_id='01dae5a273'), StructuredSolver(name='Advantage2_system1.11', graph_id='01dceba9f7')]


### Annealing parameters

- `solver_name`: D-Wave solver to use (e.g. Advantage2)
- `num_reads`: Number of samples
- `annealing_time`: Annealing time per read (μs)
- `chain_strength`: Leave `None` for auto

In [8]:
solver_name = "Advantage2_system1.11"
num_reads = 1000
annealing_time = 20
naive_chain_strength = None  # Auto
dc_chain_strength = None

In [9]:
# Sample on D-Wave with NAIVE BQM (embedding, annealing, result retrieval)
naive_result = sampling_with_dwave(
    solver_name,
    naive_bqm,
    num_reads=num_reads,
    annealing_time=annealing_time,
    chain_strength=naive_chain_strength,
)

In [10]:
# Same sampling with DIVIDE_AND_CONQUER BQM
dc_result = sampling_with_dwave(
    solver_name,
    dc_bqm,
    num_reads=num_reads,
    annealing_time=annealing_time,
    chain_strength=dc_chain_strength,
)

## Evaluating and comparing results

For both formulations we compare:

- **Feasible**: Whether the best sample satisfies row-sum and column-sum constraints
- **Compatible**: Whether the best sample satisfies incompatible-pair constraints
- **Chain Strength / Embedding Qubits / Chain Break Rate**: Embedding quality and chain break rate
- **Best Solution Energy / Average Energy**: Energy of the solutions (lower is better for the objective)

In [11]:
# Physical qubits after embedding and chain strength (auto-set values)
naive_embedding = sum([len(v) for v in naive_result.info["embedding_context"]["embedding"].values()])
naive_chain_strength = naive_result.info["embedding_context"]["chain_strength"]
dc_embedding = sum([len(v) for v in dc_result.info["embedding_context"]["embedding"].values()])
dc_chain_strength = dc_result.info["embedding_context"]["chain_strength"]

# Feasibility of best sample (row/column sums and incompatible pairs)
naive_first = naive_result.first.sample
dc_first = dc_result.first.sample
is_constrained_naive = check_feasibility(naive_first, num_days, num_workers, shifts_per_day, shifts_per_worker)
is_compatible_naive = check_incompatible_pairs(naive_first, num_days, num_workers, incompatible_pairs)
is_constrained_dc = check_feasibility(dc_first, num_days, num_workers, shifts_per_day, shifts_per_worker)
is_compatible_dc = check_incompatible_pairs(dc_first, num_days, num_workers, incompatible_pairs)

naive_df = naive_result.to_pandas_dataframe()
dc_df = dc_result.to_pandas_dataframe()
naive_energy_average = naive_df["energy"].mean()
dc_energy_average = dc_df["energy"].mean()

# Print comparison table
print(f"{'Metric':<20} | {'Naive':<10} | {'D&C':<10}")
print("-" * 46)
print(f"{'Feasible':<20} | {is_constrained_naive!s:<10} | {is_constrained_dc!s:<10}")
print(f"{'Compatible':<20} | {is_compatible_naive!s:<10} | {is_compatible_dc!s:<10}")
print(f"{'Chain Strength':<20} | {f'{naive_chain_strength:.3f}':<10} | {f'{dc_chain_strength:.3f}':<10}")
print(f"{'Embedding Qubits':<20} | {naive_embedding:<10} | {dc_embedding:<10}")
print(
    f"{'Chain Break Rate':<20} | {f'{calculate_chain_break_rate(naive_df):.3f}':<10} | {f'{calculate_chain_break_rate(dc_df):.3f}':<10}"
)
print(f"{'Best Solution Energy':<20} | {naive_result.first.energy:<10} | {dc_result.first.energy:<10}")
print(f"{'Average Energy':<20} | {f'{naive_energy_average:.3f}':<10} | {f'{dc_energy_average:.3f}':<10}")

Metric               | Naive      | D&C       
----------------------------------------------
Feasible             | True       | True      
Compatible           | True       | True      
Chain Strength       | 58.372     | 12.934    
Embedding Qubits     | 349        | 990       
Chain Break Rate     | 0.019      | 0.004     
Best Solution Energy | -39.0      | -44.0     
Average Energy       | 76.560     | 27.559    


---

The comparison table shows that which formulation (NAIVE vs DIVIDE_AND_CONQUER) is better can depend on problem size and parameters. See `create_scheduling_problem_bqm` in `examples/shift_scheduling/problem.py` to confirm that sparse-qubo lets you compare formulations by simply switching `NetworkType` when passing "exactly k selected" constraints to `create_constraint_dwave`.